In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("project").getOrCreate()

In [ ]:
# Load the Chipotle dataset into a Spark DataFrame
customer_data_path = "./Data/olist_customers_dataset.csv"  # Replace with the actual path
order_item_path = "./Data/olist_order_items_dataset.csv"
order_payment_path = "./Data/olist_order_payments_dataset.csv"
order_review_path = './Data/olist_order_reviews_dataset.csv'
product_path = './Data/olist_products_dataset.csv'
seller_path = './Data/olist_sellers_dataset.csv'
geolocation_path = './Data/olist_geolocation_dataset.csv'
orders_path = './Data/olist_orders_dataset.csv'

customer_df = spark.read.csv(customer_data_path, header=True, inferSchema=True)
order_item_df = spark.read.csv(order_item_path, header=True, inferSchema=True)
order_payment_df = spark.read.csv(order_payment_path, header=True, inferSchema=True)
order_review_df = spark.read.csv(order_review_path, header=True, inferSchema=True)
seller_df = spark.read.csv(seller_path, header=True, inferSchema=True)
product_df = spark.read.csv(product_path, header=True, inferSchema=True)
geoloacation_df = spark.read.csv(geolocation_path, header=True, inferSchema= True)
orders_df = spark.read.csv(orders_path, header=True, inferSchema= True)